# 进程安全生成器

[https://www.jiqizhixin.com/articles/2019-05-07-18](https://www.jiqizhixin.com/articles/2019-05-07-18)


In [ ]:
from skimage.io import imread
from skimage.transform import resize
import numpy as np

# Here, `x_set` is list of path to the images
# and `y_set` are the associated classes.

class CIFAR10Sequence(Sequence):

    def __init__(self, x_set, y_set, batch_size=32):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        #计算每一个epoch的迭代次数
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        #生成每个batch数据，这里就根据自己对数据的读取方式进行发挥了
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]

        return np.array([
            resize(imread(file_name), (200, 200))
               for file_name in batch_x]), np.array(batch_y)

# 数据生成器
# training_generator = CIFAR10Sequence(x,y)
# model.fit_generator(training_generator, epochs=50,max_queue_size=10,workers=1)

# 获得中间变量

In [ ]:
class Normal(Layer):
    def __init__(self, **kwargs):
        super(Normal, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(1,),
                                      initializer='zeros',
                                      trainable=True)
        self.built = True
    def call(self, x):
        self.x_normalized = K.l2_normalize(x, -1)
        return self.x_normalized * self.kernel
    
x_in = Input(shape=(784,))
x = x_in

x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
y = x
x = Dropout(0.2)(x)
x = Dense(num_classes, activation='softmax')(x)
normal = Normal()
x = normal(x)
x = Dense(num_classes, activation='softmax')(x)

model = Model(x_in, x)

# 作为一个新模型
# y 必须是某个层的输出，不能是随意一个张量。 
model2 = Model(x_in, y)

# K.function！允许是任意张量！返回的 fn 是一个具有函数功能的对象， 
# fn([x_test]) 就相当于：sess.run(normal.x_normalized, feed_dict={x_in: x_test})
fn = K.function([x_in], [normal.x_normalized])

# 权重滑动平均

假设每次优化器的更新为：

![](https://image.jiqizhixin.com/uploads/editor/e8073a2b-8a10-40f0-8ba9-7ee3bd4c05e7/640.png)

滑动平均则是维护一组新的新的变量 Θ：

![](https://image.jiqizhixin.com/uploads/editor/82608c10-6a0e-47ff-a0d5-05414ea1e3d4/640.png)

其中 α 是一个接近于 1 的正常数，称为“衰减率（decay rate）”

权重滑动平均不改变优化器的走向，只不过它降优化器的优化轨迹上的点做了平均后，作为最终的模型权重。 

In [ ]:
class ExponentialMovingAverage:
    """对模型权重进行指数滑动平均。
    用法：在model.compile之后、第一次训练之前使用；
    先初始化对象，然后执行inject方法。
    """
    def __init__(self, model, momentum=0.9999):
        self.momentum = momentum
        self.model = model
        self.ema_weights = [K.zeros(K.shape(w)) for w in model.weights]
    def inject(self):
        """添加更新算子到model.metrics_updates。
        """
        self.initialize()
        for w1, w2 in zip(self.ema_weights, self.model.weights):
            op = K.moving_average_update(w1, w2, self.momentum)
            self.model.metrics_updates.append(op)
    def initialize(self):
        """ema_weights初始化跟原模型初始化一致。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        K.batch_set_value(zip(self.ema_weights, self.old_weights))
    def apply_ema_weights(self):
        """备份原模型权重，然后将平均权重应用到模型上去。
        """
        self.old_weights = K.batch_get_value(self.model.weights)
        ema_weights = K.batch_get_value(self.ema_weights)
        K.batch_set_value(zip(self.model.weights, ema_weights))
    def reset_old_weights(self):
        """恢复模型到旧权重。
        """
        K.batch_set_value(zip(self.model.weights, self.old_weights))
        
# 使用方法很简单：

EMAer = ExponentialMovingAverage(model) # 在模型compile之后执行
EMAer.inject() # 在模型compile之后执行

model.fit(x_train, y_train) # 训练模型

# 训练完成后：

EMAer.apply_ema_weights() # 将EMA的权重应用到模型中
model.predict(x_test) # 进行预测、验证、保存等操作

EMAer.reset_old_weights() # 继续训练之前，要恢复模型旧权重。还是那句话，EMA不影响模型的优化轨迹。
model.fit(x_train, y_train) # 继续训练

# 引入了 K.moving_average_update 操作，
# 并且插入到 model.metrics_updates 中，
# 在训练过程中，模型会读取并执行 model.metrics_updates 的所有算子，
# 从而完成了滑动平均。